# Crecimiento Facebook + Suicidios

### Importo librerias

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import mysql.connector as conn
from selenium import webdriver
import time

### Funciones

In [2]:
def crear_db(nombre, user='root', password='password', check=True):
    
    crea_db=conn.connect(host='localhost', user=user, passwd=password)

    cursor=crea_db.cursor()

    cursor.execute(f'drop database if exists {nombre}')
    cursor.execute(f'create database {nombre}')
    
    print('Hecho, base de datos creada. ;)')
    
    if check:
        cursor.execute('show databases')
        for x in cursor:
            print(x)

In [3]:
def crear_tabla_desde_df(df, nombre_tabla, nombre_db, user='root', password='password', check=True, drop=False):
    
    db=conn.connect(host='localhost', user=user, passwd=password, database=nombre_db)
    cursor=db.cursor()
    
    if drop:
        cursor.execute(f'drop table if exists {nombre_tabla}')
    
    tabla=f"create table {nombre_tabla}(id int, {' varchar(200),'.join(df.columns)} varchar(200));"
    print(tabla)
    
    cursor.execute(tabla)
    
    print(f'Hecho, tabla {nombre_tabla} creada. ;)')
    
    if check:
        cursor.execute('show tables')
        for x in cursor:
            print(x)

In [4]:
def insertar_datos(df, nombre_tabla, nombre_db, user='root', password='password', check=True):
    
    db=conn.connect(host='localhost', user=user, passwd=password, database=nombre_db)
    cursor=db.cursor()
    
    for i in range(len(df)):
    
        insert_query='insert into {} (id, {}) values {};'\
                     .format(nombre_tabla, ','.join(df.columns), tuple(np.insert(df.iloc[i].values, 0, i)))

        cursor.execute(insert_query)

    db.commit()
    
    print(f'Hecho, datos dentro de tabla {nombre_tabla}. ;)')
    print()
    print(insert_query)

In [5]:
def leer_sql(query, nombre_tabla, nombre_db, user='root', password='password'):
    
    db=conn.connect(host='localhost', user=user, passwd=password, database=nombre_db)
    
    data=pd.read_sql(query, db)
    
    return data

### CSV Peliculas

In [6]:
df=pd.read_csv('MCU_movies_Dataset.csv', )

df.head()


,Id,Title,Phase,Genres,Release Date,Director,ScreenWriter,Producer,Cast,Plot,Language,Filming Locations,Status,Movie Rating,Review Rating,Movie Run Time,Budget,Box office
0,1,Iron Man,Phase One,"Action, Adventure, Sci-Fi",02-05-2008,Jon Favreau,Mark Fergus & Hawk Ostby�and�Art Marcum & Matt...,Avi Arad and Kevin Feige,"Robert Downey Jr., Gwyneth Paltrow, Terrence H...","After being held captive in an Afghan cave, bi...","English, Persian, Urdu, Arabic, Hungarian","Palmdale Regional Airport, Palmdale, Californi...",Released,PG-13,7.9,126 min,140000000.0,585174222.0
1,2,The Incredible Hulk,Phase One,"Action, Adventure, Sci-Fi",13-06-2008,Louis Leterrier,Zak Penn,"Avi Arad, Gale Anne Hurd and Kevin Feige","Edward Norton, Liv Tyler, Tim Roth, William Hurt","Bruce Banner, a scientist on the run from the ...","English, Portuguese, Spanish","Cherry Street Bridge, Toronto, Ontario, Canada",Released,PG-13,6.7,112 min,150000000.0,263427551.0
2,3,Iron Man 2,Phase One,"Action, Adventure, Sci-Fi",07-05-2010,Jon Favreau,Justin Theroux,Kevin Feige,"Robert Downey Jr., Mickey Rourke, Gwyneth Palt...",With the world now aware of his identity as Ir...,"English, French, Russian","D.C. Stages, 1360 East 6th Street, Downtown, L...",Released,PG-13,7.0,124 min,200000000.0,623933331.0
3,4,Thor,Phase One,"Action, Adventure, Fantasy, Sci-Fi",06-05-2011,Kenneth Branagh,Ashley Edward Miller�&�Zack Stentz�and�Don Payne,Kevin Feige,"Chris Hemsworth, Anthony Hopkins, Natalie Port...",The powerful but arrogant god Thor (Chris Hems...,English,"Galisteo, New Mexico, USA",Released,PG-13,7.0,115 min,150000000.0,449326618.0
4,5,Captain America: The First Avenger,Phase One,"Action, Adventure, Sci-Fi",22-07-2011,Joe Johnston,Christopher Markus & Stephen McFeely,Kevin Feige,"Chris Evans, Hugo Weaving, Samuel L. Jackson, ...","Steve Rogers, a rejected military soldier tran...","English, Norwegian, French","Stanley Dock, Liverpool, Merseyside, England, UK",Released,PG-13,6.9,124 min,140000000.0,370569774.0


In [7]:
df2 = df.drop(['Id','Genres','Phase','Review Rating','Release Date','ScreenWriter','Producer','Plot','Language','Filming Locations','Box office','Status','Movie Rating','Budget'], axis=1)

In [8]:
df2.head() #columnas borradas

,Title,Director,Cast,Movie Run Time
0,Iron Man,Jon Favreau,"Robert Downey Jr., Gwyneth Paltrow, Terrence H...",126 min
1,The Incredible Hulk,Louis Leterrier,"Edward Norton, Liv Tyler, Tim Roth, William Hurt",112 min
2,Iron Man 2,Jon Favreau,"Robert Downey Jr., Mickey Rourke, Gwyneth Palt...",124 min
3,Thor,Kenneth Branagh,"Chris Hemsworth, Anthony Hopkins, Natalie Port...",115 min
4,Captain America: The First Avenger,Joe Johnston,"Chris Evans, Hugo Weaving, Samuel L. Jackson, ...",124 min


In [9]:
 #pongo el titulo de la pelicula como indice

In [10]:
df2 = df2.rename(columns={'Movie Run Time':'Lenght'}) #Cambio nombre de columna

### Dataframe Titulo/Director/Cast/Lenght

In [11]:
df2.head()

,Title,Director,Cast,Lenght
0,Iron Man,Jon Favreau,"Robert Downey Jr., Gwyneth Paltrow, Terrence H...",126 min
1,The Incredible Hulk,Louis Leterrier,"Edward Norton, Liv Tyler, Tim Roth, William Hurt",112 min
2,Iron Man 2,Jon Favreau,"Robert Downey Jr., Mickey Rourke, Gwyneth Palt...",124 min
3,Thor,Kenneth Branagh,"Chris Hemsworth, Anthony Hopkins, Natalie Port...",115 min
4,Captain America: The First Avenger,Joe Johnston,"Chris Evans, Hugo Weaving, Samuel L. Jackson, ...",124 min


### Segundo CSV

In [12]:
ft =pd.read_csv('fuente.csv')

ft.head(1)

,Uniq Id,Crawl Timestamp,Pageurl,Title,Genres,Release Date,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,Filming Locations,Budget,Collection,Collection Date
0,bdce103d6c8ee8206477f5ef43cdd6cf,2019-05-29 12:32:00 +0000,https://www.imdb.com/title/tt6791350/,Guardians of the Galaxy Vol. 3 (2021) - IMDb,NaN,NaN,NaN,NaN,NaN,Directed by James Gunn. With Elizabeth Debicki...,NaN,English,NaN,NaN,NaN,NaN


In [13]:
ft = ft.drop(['Movie Run Time','Uniq Id','Crawl Timestamp','Pageurl','Genres','Release Date','Movie Rating','Review Rating','Plot','Cast','Language','Filming Locations','Collection Date'], axis = 1 )

In [14]:
ft.head()

,Title,Budget,Collection
0,Guardians of the Galaxy Vol. 3 (2021) - IMDb,NaN,NaN
1,Iron Man 2 (2010) - IMDb,"$200,000,000","$623,933,331,"
2,Guardians of the Galaxy (2014) - IMDb,"$170,000,000","$774,176,600"
3,Black Panther (2018) - IMDb,"$200,000,000","$1,347,071,259"
4,Iron Man 3 (2013) - IMDb,"$200,000,000","$1,215,439,994"


In [15]:
import re

In [16]:
ft.Title = [i.replace('- IMDb','') for i in ft.Title]

In [17]:
lst = []

patron = '\(([^\)]+)\)'

for e in ft.Title:
    
    lst.append(re.sub(patron,'', e))
    


In [18]:

lst = [i.strip() for i in lst]

In [19]:
ft.Title = lst

In [20]:
ft.head()

,Title,Budget,Collection
0,Guardians of the Galaxy Vol. 3,NaN,NaN
1,Iron Man 2,"$200,000,000","$623,933,331,"
2,Guardians of the Galaxy,"$170,000,000","$774,176,600"
3,Black Panther,"$200,000,000","$1,347,071,259"
4,Iron Man 3,"$200,000,000","$1,215,439,994"


In [21]:
df2.head()

,Title,Director,Cast,Lenght
0,Iron Man,Jon Favreau,"Robert Downey Jr., Gwyneth Paltrow, Terrence H...",126 min
1,The Incredible Hulk,Louis Leterrier,"Edward Norton, Liv Tyler, Tim Roth, William Hurt",112 min
2,Iron Man 2,Jon Favreau,"Robert Downey Jr., Mickey Rourke, Gwyneth Palt...",124 min
3,Thor,Kenneth Branagh,"Chris Hemsworth, Anthony Hopkins, Natalie Port...",115 min
4,Captain America: The First Avenger,Joe Johnston,"Chris Evans, Hugo Weaving, Samuel L. Jackson, ...",124 min


In [22]:
final = pd.merge(df2, ft, left_on='Title',right_on='Title')

In [23]:
final

,Title,Director,Cast,Lenght,Budget,Collection
0,Iron Man,Jon Favreau,"Robert Downey Jr., Gwyneth Paltrow, Terrence H...",126 min,"$140,000,000","$585,174,222,"
1,The Incredible Hulk,Louis Leterrier,"Edward Norton, Liv Tyler, Tim Roth, William Hurt",112 min,"$150,000,000","$263,427,551"
2,Iron Man 2,Jon Favreau,"Robert Downey Jr., Mickey Rourke, Gwyneth Palt...",124 min,"$200,000,000","$623,933,331,"
3,Thor,Kenneth Branagh,"Chris Hemsworth, Anthony Hopkins, Natalie Port...",115 min,"$150,000,000","$449,326,618"
4,Captain America: The First Avenger,Joe Johnston,"Chris Evans, Hugo Weaving, Samuel L. Jackson, ...",124 min,"$140,000,000","$370,569,774"
5,The Avengers,Joss Whedon,"Robert Downey Jr., Chris Evans, Scarlett Johan...",143 min,"$220,000,000","$1,519,557,910"
6,Iron Man 3,Shane Black,"Robert Downey Jr., Guy Pearce, Gwyneth Paltrow...",130 min,"$200,000,000","$1,215,439,994"
7,Thor: The Dark World,Alan Taylor,"Chris Hemsworth, Natalie Portman, Tom Hiddlest...",112 min,"$170,000,000","$644,783,140"
8,Captain America: The Winter Soldier,Anthony and Joe Russo,"Chris Evans, Samuel L. Jackson, Scarlett Johan...",136 min,"$170,000,000","$714,766,572"
9,Guardians of the Galaxy,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",121 min,"$170,000,000","$774,176,600"


In [24]:
### Scrapeo duración valoración

In [25]:
from selenium import webdriver

from webdriver_manager.chrome import ChromeDriverManager

import time

In [26]:
PATH= ChromeDriverManager().install()

url = 'https://www.imdb.com/list/ls063256988/'

driver = webdriver.Chrome(PATH)


driver.get(url)

time.sleep(4)

dev = driver.find_element_by_xpath('//*[@id="main"]/div/div[3]')

punt = [e.text for e in dev.find_elements_by_class_name('ipl-rating-star__rating') if e.text != '']

punt = [ e for e in punt if e != 'Rate']

driver.quit()



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/Users/alexmendezsomoza/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [27]:
punt

['8,4',
 '7,7',
 '8,4',
 '8',
 '7,6',
 '7,4',
 '7,8',
 '7,3',
 '7,5',
 '8',
 '7',
 '7,5',
 '7,9',
 '6,9',
 '7,3',
 '7,9',
 '7,3',
 '7,1',
 '6,9',
 '6,9',
 '7']

In [28]:
nombres = ['Avengers: Infinity War',
 'Captain America: The Winter Soldier',
 'Avengers: Endgame',
 'Guardians of the Galaxy',
 'Guardians of the Galaxy Vol. 2',
 'Spider-Man: Homecoming',
 'Captain America: Civil War',
 'Ant-Man',
 'Doctor Strange',
 'The Avengers',
 'Ant-Man and the Wasp',
 'Spider-Man: Far From Home',
 'Iron Man',
 'Captain Marvel',
 'Avengers: Age of Ultron',
 'Thor: Ragnarok',
 'Black Panther',
 'Iron Man 3',
 'Captain America: The First Avenger',
 'Thor: The Dark World',
 'Thor']

In [29]:
nota = pd.DataFrame(zip(nombres,punt), columns = ('Title','imdb_score'))

In [30]:
nota

,Title,imdb_score
0,Avengers: Infinity War,"8,4"
1,Captain America: The Winter Soldier,"7,7"
2,Avengers: Endgame,"8,4"
3,Guardians of the Galaxy,8
4,Guardians of the Galaxy Vol. 2,"7,6"
5,Spider-Man: Homecoming,"7,4"
6,Captain America: Civil War,"7,8"
7,Ant-Man,"7,3"
8,Doctor Strange,"7,5"
9,The Avengers,8


In [31]:
top = pd.merge(final, nota, left_on='Title',right_on='Title')

In [32]:
top = top.reset_index()

In [33]:
top.columns

Index(['index', 'Title', 'Director', 'Cast', 'Lenght', 'Budget', 'Collection',
       'imdb_score'],
      dtype='object')

In [34]:
top.columns

Index(['index', 'Title', 'Director', 'Cast', 'Lenght', 'Budget', 'Collection',
       'imdb_score'],
      dtype='object')

In [35]:
top.Budget = top.Budget.fillna('Unknown')

In [36]:
top.Budget = [i.replace('$','') for i in top.Budget]
top.Collection = [i.replace('$','') for i in top.Collection]
top.Budget = [i.strip() for i in top.Budget]
top.Collection = [i.strip() for i in top.Collection]

In [37]:
#top.columns = [c[0] for c in top.columns]

In [38]:
top = top.drop(['index'], axis=1)

In [39]:
top.columns

Index(['Title', 'Director', 'Cast', 'Lenght', 'Budget', 'Collection',
       'imdb_score'],
      dtype='object')

In [40]:
## Creo Base de Datos y Tabla en SQL

In [41]:
top.columns = [c.lower() for c in top.columns]

In [42]:
import mysql.connector as conn

In [43]:
crea_db=conn.connect(host='localhost', user='root', passwd='123456')
cursor=crea_db.cursor()
cursor.execute('drop database if exists Marvel')
cursor.execute('create database Marvel')

In [44]:
cursor.execute('show databases')
for x in cursor:
    print(x)

('information_schema',)
('Marvel',)
('mydb',)
('mysql',)
('performance_schema',)
('productos',)
('publications',)
('Sharks',)


In [45]:
db=conn.connect(host='localhost', user='root', passwd='123456', database='Marvel')

cursor=db.cursor()

In [46]:
top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       20 non-null     object
 1   director    20 non-null     object
 2   cast        20 non-null     object
 3   lenght      20 non-null     object
 4   budget      20 non-null     object
 5   collection  20 non-null     object
 6   imdb_score  20 non-null     object
dtypes: object(7)
memory usage: 1.2+ KB


In [47]:
cursor.execute('drop table if exists marvel_movies')   # borra la tabla si existe
tabla='''
        create table marvel_movies(
            
            id int,
            title varchar(500),
            director varchar(500),
            cast varchar(500),
            lenght varchar(500),
            budget varchar(500),
            collection varchar(500),
            imdb_score varchar(500)
            
        );
'''

cursor.execute (tabla)

In [48]:
cursor.execute('show tables')
for x in cursor:
    print(x)

('marvel_movies',)


In [49]:
for i in range(len(top)):
    
    
    insert_query="insert into {} ({}) values {};"\
                    .format('marvel_movies', ','.join(top.columns), tuple(top.iloc[i].values))
    
    cursor.execute(insert_query)
    
db.commit()